# Research Track 2 - Assignment 2
## Implementation of a user interface with ROS in a Jupyter notebook for Assignment 2

## Imorting necessary libraries and variables

In [ ]:
# ROS libraries
import rospy
from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
import actionlib
import actionlib.msg
from rt1_a2_2023.msg import PlanningAction, PlanningGoal
from rt1_a2_2023.msg import TargetPosition, Abort, RobotPositionVelocity
from std_srvs.srv import *
from rt1_a2_2023.srv import LastTarget, RobotToTarget

# ipywidgets libraries
import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox 
from IPython.display import display 
from IPython.display import clear_output

################################

# global variables
global temp_status
global have_goal
global goal_reached

# Initialize the publisher
global publisher
publisher = rospy.Publisher("/RobotPositionVelocity", RobotPositionVelocity, queue_size=1)

global action_client

global now_x
global now_y
x_arrey = []
y_arrey = []

canceled_goal_x = []
canceled_goal_y = []

new_given_goal_x = []
new_given_goal_y = []

number_reached_goals = 0

################################


## interface logic

In [ ]:

# def interface_behaviour(selector):
#     if selector:
#         x_coord.disabled=False
#         y_coord.disabled=False
#         submit_b.disabled=False
#         cancel_b.disabled=True

#     else:
#         x_coord.disabled=True
#         y_coord.disabled=True
#         submit_b.disabled=True
#         cancel_b.disabled=False

## callback function for the subscriber

In [ ]:
def publish_message(msg):
    global publisher
    # get the current position of the robot from the msg in the topic /odom.
    position = msg.pose.pose.position
    linear_velocity = msg.twist.twist.linear

    now_x = position.x
    now_y = position.y

    x_arrey.append(now_x)
    y_arrey.append(now_y)

    #define the message that will be published
    pos_vel = RobotPositionVelocity()
    pos_vel.x = position.x
    pos_vel.y = position.y
    pos_vel.v_x = linear_velocity.x
    pos_vel.v_y = linear_velocity.y

    # publish the message
    publisher.publish(pos_vel)

## Cancel button callback function

In [ ]:
def cancel_button_clicked_callback(b):
    with output:
        if have_goal:
            canceled_goal_x.append(rospy.get_param('/des_pos_x'))
            canceled_goal_y.append(rospy.get_param('/des_pos_y'))
            # cancel the goal
            have_goal = 0
            action_client.cancel_goal()
            rospy.loginfo("Goal canceled")
            print("Target cancelled")
        elif not have_goal:
            rospy.loginfo("No goal to cancel")

## Change button callback function

In [ ]:
def change_button_clicked_callback(b):
    with output:
        if have_goal:
            # cancel the goal first
            have_goal = 0
            action_client.cancel_goal()
            rospy.loginfo("Goal canceled")
            # get the new goal from the client
            rospy.set_param('/des_pos_x', c1_slider.value)
            rospy.set_param('/des_pos_y', c2_slider.value)
            # change the goal and send it to the action server
            new_target_x = c1_slider.value
            new_target_y = c2_slider.value
            rospy.loginfo("New goal is: x = %f, y = %f", new_target_x, new_target_y)
            goal_new = rt1_a2_2023.msg.PlanningGoal()
            goal_new.target_pose.pose.position.x = new_target_x
            goal_new.target_pose.pose.position.y = new_target_y
            new_given_goal_x.append(new_target_x)
            new_given_goal_y.append(new_target_y)

            # send new goal to the action server
            action_client.send_goal(goal_new)
            have_goal = 1

            print("New target set: X =",  goal_new.target_pose.pose.position.x, "Y =", goal_new.target_pose.pose.position.y)

## Info button callback function

In [ ]:
def info_button_clicked_callback(b):
    with output:
        print("Current position: X =", now_x, "Y =", now_y)
        given_goals_list = list(zip(new_given_goal_x, new_given_goal_y))
        print("Given goals: ", given_goals_list)
        canceled_goals_list = list(zip(canceled_goal_x, canceled_goal_y))
        print("Canceled goals: ", canceled_goals_list)

## Initialize the node and subscribe to the topic /odom channel to get the current position of the robot

In [ ]:
# Initialize the node
rospy.init_node('DefineActionClient')

# log the initialization of the node
rospy.loginfo("Node initialized successfully")

# Initialize the subscriber
global subscriber
subscriber = rospy.Subscriber("/odom", Odometry, publish_message)

# Initialize the action client
action_client = actionlib.SimpleActionClient('/reaching_goal', rt1_a2_2023.msg.PlanningAction)
action_client.wait_for_server()
rospy.loginfo("Action client initialized successfully")

now_x = rospy.get_param('/des_pos_x')
now_y = rospy.get_param('/des_pos_y')

rospy.loginfo("Current position: x = %f, y = %f", now_x, now_y)